In [1]:
%reset -fs
import bs4
from bs4 import BeautifulSoup as bs
import pandas as pd
import numpy as np
import requests
import pickle
import re

<div class="alert alert-block alert-danger">

 <span style="color:red ; font-size:1.2em" > NOTE TO CODE REVIEWER: At 11am on Feb 22, my access to the web site I used to scrape my data was blocked. The code in this notebook is fine since it uses pickled files created before the block, unless the other notebooks in my submission were run and the pickled files they generate altered.</span>
</div>

<div class="alert alert-block alert-info">

</div>

<div class="alert alert-block alert-info">
<h3> In this notebook:</h3>
<br>
<span style="font-size:1.2em" >
    <ul>
        <li>I combine the data from the final dataframes from all years into a final all years dataframe</li>
        <li> Drop columns that at this point I judge to be not useful</li>
        <li> Drop all rows with Nan values and store in a separate dataframe</li>
        <li>A final dataframe for all years, one with Nan and another with all Nan removed is pickled</li>
    </ul>
</span>
</div>

<div class="alert alert-block alert-info">

</div>

In [2]:
#read in final dataframes from all years

df_2019 = pd.read_pickle('../data_2019/final_2019_pk')
df_2018 = pd.read_pickle('../data_2018/final_2018_pk')
df_2017 = pd.read_pickle('../data_2017/final_2017_pk')
df_2016 = pd.read_pickle('../data_2016/final_2016_pk')
df_2015 = pd.read_pickle('../data_2015/final_2015_pk')

# df_2019
# df_2018
# df_2017
df_2015

,Rank,Movie,ReleaseDate,Revenues,Tickets_Sold,Legs,Budget,WBO_to_Budget_Ratio,Theater_Count,Average_Run,...,Samuel L. Jackson,"Robert Downey, Jr.",Scarlett Johansson,Tom Hanks,Bradley Cooper,Harrison Ford,Chris Evans,Tom Cruise,Chris Hemsworth,Zoe Saldana
0,1,Star Wars Ep. VII: The Forc…,"Dec 18, 2015",742208942,88043765,3.78,306000000.0,6.7,4134.0,9.2,...,0,0,0,0,0,1,0,0,0,0
1,2,Jurassic World,"Jun 12, 2015",652270625,77374926,3.12,215000000.0,7.8,4274.0,8.5,...,0,0,0,0,0,0,0,0,0,0
2,3,Avengers: Age of Ultron,"May 1, 2015",459005868,54449094,2.40,365000000.0,3.8,4276.0,7.0,...,1,1,1,0,0,0,1,0,1,0
3,4,Inside Out,"Jun 19, 2015",356461711,42284900,3.94,175000000.0,4.9,3946.0,8.9,...,0,0,0,0,0,0,0,0,0,0
4,5,Furious 7,"Apr 3, 2015",353007020,41875091,2.40,190000000.0,8.0,4003.0,6.9,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
807,808,Lou!,"Apr 24, 2015",88,10,1.00,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
808,809,Paranoid Girls,"Nov 19, 2015",54,6,1.00,NaN,NaN,1.0,1.0,...,0,0,0,0,0,0,0,0,0,0
809,810,Memphis,"Sep 5, 2014",54,6,1.00,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
810,811,The Historian,"Nov 7, 2014",36,4,84.80,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0


In [3]:
#Create an all years dataframe and concat dataframes from all years

frames = (df_2019, df_2018, df_2017, df_2016, df_2015)
all_years_df = pd.concat(frames, ignore_index = True)

In [4]:
#Sort dataframe based on revenues.
#Rename rank column

all_years_df.sort_values('Revenues', ascending = False ,
                         inplace = True, ignore_index = True)
all_years_df.rename(columns = {'Rank': 'Rank_in_release_yr'}, inplace = True)

all_years_df

,Rank_in_release_yr,Movie,ReleaseDate,Revenues,Tickets_Sold,Legs,Budget,WBO_to_Budget_Ratio,Theater_Count,Average_Run,...,Samuel L. Jackson,"Robert Downey, Jr.",Scarlett Johansson,Tom Hanks,Bradley Cooper,Harrison Ford,Chris Evans,Tom Cruise,Chris Hemsworth,Zoe Saldana
0,1,Avengers: Endgame,"Apr 26, 2019",858373000,93708843,2.40,400000000.0,7.0,4662.0,8.1,...,1,1,1,0,1,0,1,0,1,1
1,1,Star Wars Ep. VII: The Forc…,"Dec 18, 2015",742208942,88043765,3.78,306000000.0,6.7,4134.0,9.2,...,0,0,0,0,0,1,0,0,0,0
2,1,Black Panther,"Feb 16, 2018",700059566,76845177,3.47,200000000.0,6.7,4020.0,9.9,...,0,0,0,0,0,0,0,0,0,0
3,2,NaN,"Apr 27, 2018",678815482,74513225,2.63,300000000.0,6.8,4474.0,7.7,...,1,1,1,0,1,0,1,0,1,1
4,2,Jurassic World,"Jun 12, 2015",652270625,77374926,3.12,215000000.0,7.8,4274.0,8.5,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3763,808,Lou!,"Apr 24, 2015",88,10,1.00,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
3764,809,Paranoid Girls,"Nov 19, 2015",54,6,1.00,NaN,NaN,1.0,1.0,...,0,0,0,0,0,0,0,0,0,0
3765,810,Memphis,"Sep 5, 2014",54,6,1.00,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
3766,811,The Historian,"Nov 7, 2014",36,4,84.80,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0


In [5]:
all_years_df.columns

Index(['Rank_in_release_yr', 'Movie', 'ReleaseDate', 'Revenues',
       'Tickets_Sold', 'Legs', 'Budget', 'WBO_to_Budget_Ratio',
       'Theater_Count', 'Average_Run', 'Genre', 'Source', 'Creative_Type',
       'Distributor', 'All_Cast_Rating', 'Samuel L. Jackson',
       'Robert Downey, Jr.', 'Scarlett Johansson', 'Tom Hanks',
       'Bradley Cooper', 'Harrison Ford', 'Chris Evans', 'Tom Cruise',
       'Chris Hemsworth', 'Zoe Saldana'],
      dtype='object')

In [6]:
#Check unique values of categorical columns

all_years_df['Genre'].unique()

array(['Action', 'Adventure', 'Musical', 'Drama', 'Thriller/Suspense',
       'Horror', 'Comedy', 'Romantic Comedy', 'Western', 'Black Comedy',
       'Concert/Perfor…', 'Documentary', 'Multiple Genres', '', 'Reality'],
      dtype=object)

In [7]:
all_years_df['Creative_Type'].unique()

array(['Super Hero', 'Science Fiction', 'Kids Fiction', 'Fantasy',
       'Contemporary Fiction', 'Dramatization', 'Historical Fiction',
       'Factual', nan, 'Multiple Creative Types'], dtype=object)

In [8]:
all_years_df['Source']= all_years_df['Source'].str.replace('Based on ', '')

In [9]:
all_years_df['Source'].unique()

array(['Comic/Graphic Novel', 'Original Screenplay',
       'Fiction Book/Short Story', 'Remake', 'Folk Tale/Legend/Fairytale',
       'Spin-Off', 'Factual Book/Article', 'TV', 'Real Life Events',
       'Toy', 'Theme Park Ride', 'Game', 'Movie', 'Song', 'Short Film',
       'Play', 'Religious Text', 'vs. Source Code', 'Musical or Opera',
       'Web Series', nan, 'Compilation', 'Musical Group'], dtype=object)

In [10]:
all_years_df['Distributor'].unique()

array(['Walt Disney', 'Universal', 'Warner Bros.', 'Sony Pictures',
       '20th Century Fox', 'Lionsgate', 'Paramount Pictures',
       'STX Entertainment', 'MGM', 'United Artists', 'Focus Features',
       'Roadside Attractions', 'Weinstein Co.', 'Fox Searchlight',
       'Entertainment Stu…', 'A24', 'CBS Films', 'Open Road',
       'Aviron Pictures', 'FUNimation', 'Annapurna Pictures',
       'Broad Green Pictures', 'Bleecker Street', 'Neon',
       'Focus / Gramercy', 'Relativity', 'Avco Embassy',
       'Pure Flix Enterta…', 'EuropaCorp', 'Great India Films',
       'Sony Pictures Cla…', '', 'Global Road', 'Miramax/Roadside …',
       'Amazon Studios', 'Orion Pictures', 'RADiUS-TWC',
       'Magnolia Pictures', 'Quality Flix', 'Walter Reade Orga…',
       'BH Tilt', 'Greenwich', 'High Top Releasing', 'LD Entertainment',
       'UTV Communications', 'Freestyle Releasing', 'OTL Releasing',
       'Eros Entertainment', 'IFC Films', 'Atlas Distribution',
       'Bluesky Cinemas', 'Bri

In [12]:
#Drop distributor column since there are too many categories to be useful

all_years_df.drop(columns = ['Distributor'], inplace = True)

In [13]:
# Drop all rows with Nan values and store in a separate dataframe

all_years_nonan_df = all_years_df.dropna().copy(deep = True)
                      
                      
all_years_nonan_df.sort_values('Revenues',ascending = False,
                               inplace = True, ignore_index = True)


In [14]:
all_years_nonan_df

,Rank_in_release_yr,Movie,ReleaseDate,Revenues,Tickets_Sold,Legs,Budget,WBO_to_Budget_Ratio,Theater_Count,Average_Run,...,Samuel L. Jackson,"Robert Downey, Jr.",Scarlett Johansson,Tom Hanks,Bradley Cooper,Harrison Ford,Chris Evans,Tom Cruise,Chris Hemsworth,Zoe Saldana
0,1,Avengers: Endgame,"Apr 26, 2019",858373000,93708843,2.40,400000000.0,7.0,4662.0,8.1,...,1,1,1,0,1,0,1,0,1,1
1,1,Star Wars Ep. VII: The Forc…,"Dec 18, 2015",742208942,88043765,3.78,306000000.0,6.7,4134.0,9.2,...,0,0,0,0,0,1,0,0,0,0
2,1,Black Panther,"Feb 16, 2018",700059566,76845177,3.47,200000000.0,6.7,4020.0,9.9,...,0,0,0,0,0,0,0,0,0,0
3,2,Jurassic World,"Jun 12, 2015",652270625,77374926,3.12,215000000.0,7.8,4274.0,8.5,...,0,0,0,0,0,0,0,0,0,0
4,3,Incredibles 2,"Jun 15, 2018",608581744,66803704,3.33,200000000.0,6.2,4410.0,9.1,...,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1133,787,Skin Trade,"May 8, 2015",1242,147,2.44,9000000.0,0.0,1.0,3.0,...,0,0,0,0,0,0,0,0,0,0
1134,719,Believe,"Dec 2, 2016",975,108,1.86,3500000.0,0.3,638.0,1.7,...,0,0,0,0,0,0,0,0,0,0
1135,790,United Passions,"Jun 5, 2015",918,108,1.00,30000000.0,0.0,10.0,1.0,...,0,0,0,0,0,0,0,0,0,0
1136,733,Higher Power,"May 11, 2018",528,57,1.77,500000.0,0.0,3.0,1.0,...,0,0,0,0,0,0,0,0,0,0


In [15]:
all_years_nonan_df.dtypes

Rank_in_release_yr      object
Movie                   object
ReleaseDate             object
Revenues                 int64
Tickets_Sold             int64
Legs                   float64
Budget                 float64
WBO_to_Budget_Ratio    float64
Theater_Count          float64
Average_Run            float64
Genre                   object
Source                  object
Creative_Type           object
All_Cast_Rating        float64
Samuel L. Jackson        int64
Robert Downey, Jr.       int64
Scarlett Johansson       int64
Tom Hanks                int64
Bradley Cooper           int64
Harrison Ford            int64
Chris Evans              int64
Tom Cruise               int64
Chris Hemsworth          int64
Zoe Saldana              int64
dtype: object

In [16]:
all_years_df

,Rank_in_release_yr,Movie,ReleaseDate,Revenues,Tickets_Sold,Legs,Budget,WBO_to_Budget_Ratio,Theater_Count,Average_Run,...,Samuel L. Jackson,"Robert Downey, Jr.",Scarlett Johansson,Tom Hanks,Bradley Cooper,Harrison Ford,Chris Evans,Tom Cruise,Chris Hemsworth,Zoe Saldana
0,1,Avengers: Endgame,"Apr 26, 2019",858373000,93708843,2.40,400000000.0,7.0,4662.0,8.1,...,1,1,1,0,1,0,1,0,1,1
1,1,Star Wars Ep. VII: The Forc…,"Dec 18, 2015",742208942,88043765,3.78,306000000.0,6.7,4134.0,9.2,...,0,0,0,0,0,1,0,0,0,0
2,1,Black Panther,"Feb 16, 2018",700059566,76845177,3.47,200000000.0,6.7,4020.0,9.9,...,0,0,0,0,0,0,0,0,0,0
3,2,NaN,"Apr 27, 2018",678815482,74513225,2.63,300000000.0,6.8,4474.0,7.7,...,1,1,1,0,1,0,1,0,1,1
4,2,Jurassic World,"Jun 12, 2015",652270625,77374926,3.12,215000000.0,7.8,4274.0,8.5,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3763,808,Lou!,"Apr 24, 2015",88,10,1.00,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
3764,809,Paranoid Girls,"Nov 19, 2015",54,6,1.00,NaN,NaN,1.0,1.0,...,0,0,0,0,0,0,0,0,0,0
3765,810,Memphis,"Sep 5, 2014",54,6,1.00,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
3766,811,The Historian,"Nov 7, 2014",36,4,84.80,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0


In [17]:
#Store final all years dataframes to pickle

# all_years_df.to_pickle('all_years_df_pk')
# all_years_nonan_df.to_pickle('all_years_nonan_df_pk')